# Source Data: 
* Source = Eurostat
* URL = https://ec.europa.eu/eurostat/databrowser/view/PRC_FSC_IDX__custom_3513786/bookmark/line?lang=en&bookmarkId=002935d0-f1d1-4419-9699-db9e2c48ced6

Index selection to compare food price changes across one or several types of indices along the food supply chain: 
* Harmonised index of consumer prices (Selected)
* Producer price index
* Agricultural commodity price index
* Import price index

Different statistical units: 
* Index with 2015=100 (Selected)
* Percentage change compared with the same month of the previous year


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import colors

In [ ]:
filename = "HICP_Index2015.csv"
data = pd.read_csv(filename)

In [ ]:
# Filter for
#   indx = "Harmonised Index of Consumer Prices"
#   unit = "Index, 2015=100"
#   coicop = "Food"
filtered_data = data[
    (data["indx"] == "Harmonised index of consumer prices") &
    (data["unit"] == "Index, 2015=100") &
    (data["coicop"] == "Food")
]

# Separate data by geography
germany_data = filtered_data[filtered_data["geo"] == "Germany"][
    ["TIME_PERIOD", "OBS_VALUE"]
].copy()
greece_data = filtered_data[filtered_data["geo"] == "Greece"][
    ["TIME_PERIOD", "OBS_VALUE"]
].copy()
eu_28_data = filtered_data[
    filtered_data["geo"] == "European Union - 28 countries (2013-2020)"
][["TIME_PERIOD", "OBS_VALUE"]].copy()
eu_27_data = filtered_data[
    filtered_data["geo"] == "European Union - 27 countries (from 2020)"
][["TIME_PERIOD", "OBS_VALUE"]].copy()

# Step 2: Convert and Adjust Data
for dataset in [germany_data, greece_data, eu_28_data, eu_27_data]:
    dataset["TIME_PERIOD"] = pd.to_datetime(dataset["TIME_PERIOD"], errors="coerce")
    dataset["OBS_VALUE"] = pd.to_numeric(dataset["OBS_VALUE"], errors="coerce")
    dataset["Percentage Increase"] = dataset["OBS_VALUE"] - 100

# Step 3: Combine Data for Plotting
combined_data = pd.concat(
    [
        germany_data.assign(Region="Deutschland"),
        greece_data.assign(Region="Griechenland"),
        eu_28_data.assign(Region="EU28 (2013-2020)"),
        eu_27_data.assign(Region="EU27 (ab 2020)"),
    ]
)

In [ ]:
color = colors.bright_colors

plt.figure(figsize=(12, 8))
plt.plot(
    germany_data["TIME_PERIOD"],
    germany_data["Percentage Increase"],
    label="Deutschland",
    linestyle="--",
    color=color["Med Blue"],
)
plt.plot(
    greece_data["TIME_PERIOD"],
    greece_data["Percentage Increase"],
    label="Griechenland",
    linestyle="-",
    color=color["Orange"],
)
plt.plot(
    eu_28_data["TIME_PERIOD"],
    eu_28_data["Percentage Increase"],
    label="EU28 (inkl. GB)",
    linestyle="-.",
    color=color["Green"],
)
plt.plot(
    eu_27_data["TIME_PERIOD"],
    eu_27_data["Percentage Increase"],
    label="EU27 (eksl. GB)",
    linestyle=":",
    color=color["Purple"],
)

# Add a thick horizontal line at 2015 (y=0)
plt.axhline(y=0, color="black", linewidth=2, linestyle="--", label="Basisjahr (2015)")


# Add labels, title, and legend
plt.xlabel("Jahr")
plt.ylabel("Prozentuale Zunahme seit 2015 (%)")
plt.title("Lebensmittelpreisindex (HICP) im Vergleich")

# Add a footnote indicating the data source
plt.figtext(
    0.5,
    -0.05,
    "Datenquelle: Eurostat - https://ec.europa.eu/eurostat/cache/website/economy/food-price-monitoring/",
    wrap=True,
    horizontalalignment="center",
    fontsize=10,
)

plt.legend()    
plt.grid(True)
plt.tight_layout()

# Show the updated plot
plt.show()

In [ ]:
# Step 4: Create the Plot
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 8))
sns.lineplot(
    data=combined_data,
    x="TIME_PERIOD",
    y="Percentage Increase",
    hue="Region",
    style="Region",
    markers=True,
    dashes=False,
)
plt.axhline(y=0, color="black", linewidth=2, linestyle="--", label="Basisjahr (2015)")
plt.xlabel("Jahr")
plt.ylabel("Prozentuale Zunahme seit 2015 (%)")
plt.title("Lebensmittelpreisindex (HICP) im Vergleich")
plt.legend(title="Region")
plt.tight_layout()

plt.savefig(f"outputs/HICP_Index2015.png", dpi=300, bbox_inches="tight")
plt.savefig(f"outputs/HICP_Index2015.svg", format="svg", bbox_inches="tight")

plt.show()

In [ ]:
# Define a function to create individual plots for each region
def plot_region(
    data,
    region_name,
    color=colors.bright_colors["Med Blue"],
    language="en",
    save_plotname=None,
):
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=data,
        x="TIME_PERIOD",
        y="Percentage Increase",
        color=color,
        label=region_name,
        linewidth=2,
        markers=True,
        dashes=False,
    )

    if language == "en":
        plt.axhline(
            y=0, color="black", linewidth=2, linestyle="--", label="Base Year (2015)"
        )
        plt.xlabel("Year")
        plt.ylabel("Percentage Change since 2015 (%)")
        plt.title(f"Food Price Index: {region_name}")
    elif language == "de":
        plt.axhline(
            y=0, color="black", linewidth=2, linestyle="--", label="Basisjahr (2015)"
        )
        plt.xlabel("Jahr")
        plt.ylabel("Prozentuale Zunahme seit 2015 (%)")
        plt.title(f"Lebensmittelpreisindex: {region_name}")

    # plt.ylim([-25, 55])

    plt.legend()
    plt.tight_layout()

    if save_plotname:
        plt.savefig(f"outputs/{save_plotname}.png", dpi=300, bbox_inches="tight")
        plt.savefig(f'outputs/{save_plotname}.svg', format='svg', bbox_inches='tight')

    plt.show()

In [ ]:
# Plot for Germany
plot_region(
    germany_data,
    "Deutschland",
    language="de",
    save_plotname="HICP_Index2015_Germany"
)

# Plot for Greece
plot_region(
    greece_data,
    "Greece",
    language="en",
    save_plotname="HICP_Index2015_Greece"
)

# Plot for EU-27
plot_region(
    eu_27_data,
    "EU 27 (excl. UK)",
    language="en",
    save_plotname="HICP_Index2015_EU27",
)

# Plot for EU-28
plot_region(
    eu_28_data,
    "EU 28 (incl. UK)",
    language="en",
    save_plotname="HICP_Index2015_EU28",
)